In [1]:
#    ____       _       _
#   |  _ \  ___| |_ __ | |__   __ _  ___
#   | | | |/ _ \ | '_ \| '_ \ / _` |/ _ \
#   | |_| |  __/ | |_) | | | | (_| |  __/
#   |____/ \___|_| .__/|_| |_|\__,_|\___|
#                |_|

In [2]:
import requests
import json
from datetime import datetime as dt
import pandas as pd

## Azure AD Authentication

https://docs.microsoft.com/nl-nl/azure/active-directory/develop/howto-create-service-principal-portal

1. create Azure AD application to 'App registrations' <br>
    for example: 'azure-lab4-arm-api' <br>
    redirect: http://localhost <br>
    create 'Client secret' and save to variable azure_secret <br>
<br>    
2. assign application to a RBAC role 'Owner'<br>
   via Cost Management + Billing, select Subscription 'Free Trial'<br>
   Access Control (IAM)<br>
   Role assignments<br>
   Search by application name 'azure-lab4-arm-api'<br>
   Add role assigment to 'Owner'
   

### Azure ID's

In [3]:
# azure-lab4-arm-api

# subscription ID can be found on a resource, overview page
azure_subscription_id = 'a8682560-0e6b-4834-accf-21ff95b29125'

# app information can be found:
# Azure Active Directory
# app Registrations
# overview
azure_tenant = '3089f1b6-d716-42a9-9dd5-a580cf7b4f12'
azure_client_id = '0407876f-fcd1-4260-bfa4-4180a54beced'

# client secret only visible during creation
# copy & pase here
azure_secret = 'iV-6G@Ee=1Cw7T/C0G1RPkPP/S[W/D+r'

### Request Access Token

In [27]:
# authorize with azure

#
# tutorial
# http://exponentialcoding.com/accessing-azure-rest-api-from-python/
#
# access_token expires in 2 hours
#

url = "https://login.windows.net/%s/oauth2/token" % azure_tenant

#resource = 'https://management.core.windows.net' # Azure Classic Deployment
resource = 'https://management.azure.com' # Azure Resource Manager
parameters = {
    'resource' : resource,
    'client_id' : azure_client_id,
    'grant_type' : 'client_credentials',
    'client_secret': azure_secret
}

headers = {'Content-Type': 'application/x-www-form-urlencoded'}
response = requests.post(url, data=parameters, headers=headers)
jsdata = response.json()

print (json.dumps(jsdata, indent=4))
access_token = jsdata['access_token']

{
    "token_type": "Bearer",
    "not_before": "1565858114",
    "access_token": "eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6ImllX3FXQ1hoWHh0MXpJRXN1NGM3YWNRVkduNCIsImtpZCI6ImllX3FXQ1hoWHh0MXpJRXN1NGM3YWNRVkduNCJ9.eyJhdWQiOiJodHRwczovL21hbmFnZW1lbnQuYXp1cmUuY29tIiwiaXNzIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvMzA4OWYxYjYtZDcxNi00MmE5LTlkZDUtYTU4MGNmN2I0ZjEyLyIsImlhdCI6MTU2NTg1ODExNCwibmJmIjoxNTY1ODU4MTE0LCJleHAiOjE1NjU4NjIwMTQsImFpbyI6IjQyRmdZUEJ4WVFwSVVMM2tkSjFiNEZhMVFYOElBQT09IiwiYXBwaWQiOiIwNDA3ODc2Zi1mY2QxLTQyNjAtYmZhNC00MTgwYTU0YmVjZWQiLCJhcHBpZGFjciI6IjEiLCJpZHAiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC8zMDg5ZjFiNi1kNzE2LTQyYTktOWRkNS1hNTgwY2Y3YjRmMTIvIiwib2lkIjoiNjA0NTZjNDMtMjg2OS00Njk1LTlhMzktNzAzZTFmMzU3OWIxIiwic3ViIjoiNjA0NTZjNDMtMjg2OS00Njk1LTlhMzktNzAzZTFmMzU3OWIxIiwidGlkIjoiMzA4OWYxYjYtZDcxNi00MmE5LTlkZDUtYTU4MGNmN2I0ZjEyIiwidXRpIjoiMVJqWUVUbFhNME84bzFpV3JadlNBQSIsInZlciI6IjEuMCJ9.3IHpWJgu4t492SDbWptErBgmEALDJF4C-I4l4aalQtSmUJEBY2WSfuayz0Y1oYGz61-9ttxqxVMHJZKkHrh-j2Oy7L2iz1UrzA5kzWp

In [28]:
# access token valid for 1 hour
# access token experation time
print ('GMT %s' % (dt.fromtimestamp(float(jsdata['expires_on']))))
print ('CET %s' % (dt.fromtimestamp(float(jsdata['expires_on'])+2*3600)))

GMT 2019-08-15 09:40:14
CET 2019-08-15 11:40:14


## Azure Resource Explorer
https://resources.azure.com

copy & paste the URL to variable 'url'<br>
your Azure subscription_id will change after the Free Trial period<br>
copy & past the new URL and save to variable 'url' below

In [29]:
headers = {'Authorization': 'Bearer ' + access_token, 
           'Content-Type': 'application/json'}

In [13]:
# subscriptions

url = 'https://management.azure.com/subscriptions?api-version=2014-04-01'
response = requests.get(url, headers=headers)
print (response.status_code)
jsdata = response.json()
print (json.dumps(jsdata['value'], indent=4))


200
[
    {
        "subscriptionPolicies": {
            "locationPlacementId": "Public_2014-09-01",
            "quotaId": "FreeTrial_2014-09-01",
            "spendingLimit": "On"
        },
        "subscriptionId": "a8682560-0e6b-4834-accf-21ff95b29125",
        "state": "Enabled",
        "displayName": "Free Trial",
        "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125"
    }
]


In [118]:
# resource groups

url = "https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups?api-version=2014-04-01"
#print (url)
response = requests.get(url, headers=headers, timeout=5)
print (response.status_code)
jsdata = response.json()
print (json.dumps(jsdata['value'], indent=4))

200
[
    {
        "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/cloud-shell-storage-westeurope",
        "name": "cloud-shell-storage-westeurope",
        "location": "westeurope",
        "properties": {
            "provisioningState": "Succeeded"
        }
    },
    {
        "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail",
        "name": "delphae-freetrail",
        "location": "westeurope",
        "properties": {
            "provisioningState": "Succeeded"
        }
    },
    {
        "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/NetworkWatcherRG",
        "name": "NetworkWatcherRG",
        "location": "westeurope",
        "properties": {
            "provisioningState": "Succeeded"
        }
    }
]


In [8]:
# delphae-freetrail

url = "https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail?api-version=2014-04-01"
response = requests.get(url, headers=headers, timeout=5)
print (response.status_code)
jsdata = response.json()
print (json.dumps(jsdata, indent=4))

200
{
    "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail",
    "name": "delphae-freetrail",
    "location": "westeurope",
    "properties": {
        "provisioningState": "Succeeded"
    }
}


In [26]:
# virtual machines

url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Compute/virtualMachines?api-version=2018-10-01'
response = requests.get(url, headers=headers, timeout=5)
print (response.status_code)
jsdata = response.json()
print (json.dumps(jsdata['value'][0], indent=4))

for vm in jsdata['value']:
    print ('{:20s}{:15s}{:15s}{:10s}'.format(vm['name'], 
                           vm['location'], 
                           vm['properties']['storageProfile']['osDisk']['osType'], 
                           vm['properties']['storageProfile']['imageReference']['sku']))

200
{
    "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Compute/virtualMachines/azure-ie-ubuntu-03",
    "name": "azure-ie-ubuntu-03",
    "location": "northeurope",
    "properties": {
        "provisioningState": "Succeeded",
        "storageProfile": {
            "imageReference": {
                "version": "latest",
                "sku": "18.04-LTS",
                "publisher": "Canonical",
                "offer": "UbuntuServer"
            },
            "dataDisks": [],
            "osDisk": {
                "caching": "ReadWrite",
                "managedDisk": {
                    "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/DELPHAE-FREETRAIL/providers/Microsoft.Compute/disks/azure-ie-ubuntu-03_OsDisk_1_cecbf9731d22446296938226597069a9"
                },
                "name": "azure-ie-ubuntu-03_OsDisk_1_cecbf9731d22446296938226597069a9",
                "createOption": "Fr

In [28]:
# connections

url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/connections?api-version=2018-07-01'
response = requests.get(url, headers=headers)
print (response.status_code)
jsdata = response.json()
#print (json.dumps(jsdata['value'][0], indent=4))
print (json.dumps(jsdata['value'][0]['properties'], indent=4))

url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/connections/lab2-meraki-vpn?api-version=2018-07-01'
response = requests.get(url, headers=headers)
print (response.status_code)
jsdata = response.json()

200


In [29]:
print (json.dumps(jsdata, indent=4))

{
    "location": "westeurope",
    "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/connections/lab2-meraki-vpn",
    "name": "lab2-meraki-vpn",
    "properties": {
        "ingressBytesTransferred": 697416,
        "sharedKey": "Delphae2019",
        "localNetworkGateway2": {
            "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/localNetworkGateways/lab2-meraki-sdwan"
        },
        "provisioningState": "Succeeded",
        "connectionStatus": "Connecting",
        "virtualNetworkGateway1": {
            "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/virtualNetworkGateways/lab2-vnet-gw"
        },
        "usePolicyBasedTrafficSelectors": false,
        "expressRouteGatewayBypass": false,
        "ipsecPolicies": [],
        "routingWeight": 0,
        "egressBy

In [30]:
properties = jsdata['properties']
print (properties['connectionStatus'])
print (properties['ingressBytesTransferred'] / 1024.0 / 1024.0)
print (properties['egressBytesTransferred'] / 1024.0 / 1024.0)

Connecting
0.6651077270507812
5.55462646484375


In [31]:
# localNetworkGateways
url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/localNetworkGateways?api-version=2018-07-01'

response = requests.get(url, headers=headers)
print (response.status_code)
jsdata = response.json()
print (json.dumps(jsdata['value'][0], indent=4))


200
{
    "location": "westeurope",
    "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/localNetworkGateways/lab2-meraki-sdwan",
    "name": "lab2-meraki-sdwan",
    "properties": {
        "gatewayIpAddress": "84.31.151.136",
        "resourceGuid": "3e7c0b0a-6e51-417a-8b47-bbe8b8dd8d27",
        "provisioningState": "Succeeded",
        "localNetworkAddressSpace": {
            "addressPrefixes": [
                "192.168.64.0/24"
            ]
        }
    },
    "etag": "W/\"191f5411-3325-4403-bd60-670914c7bd61\"",
    "type": "Microsoft.Network/localNetworkGateways"
}


In [32]:
localgw = jsdata['value'][0]
print ('{:20}{}'.format(localgw['name'], localgw['properties']['localNetworkAddressSpace']))
print ('{:20}{}'.format(localgw['name'], localgw['properties']['gatewayIpAddress']))

lab2-meraki-sdwan   {'addressPrefixes': ['192.168.64.0/24']}
lab2-meraki-sdwan   84.31.151.136


In [33]:
# resources

url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resources?api-version=2017-05-10'
response = requests.get(url, headers=headers)
print (response.status_code)
jsdata = response.json()
data = jsdata['value']

df = pd.DataFrame(data)

200


In [34]:
df[['name','type','location']][df.type=='Microsoft.Compute/virtualMachines']

,name,type,location
6,azure-fr-ubuntu-07,Microsoft.Compute/virtualMachines,francecentral
7,azure-ie-ubuntu-03,Microsoft.Compute/virtualMachines,northeurope
8,azure-nl-ubuntu-01,Microsoft.Compute/virtualMachines,westeurope
9,azure-uk-ubuntu-04,Microsoft.Compute/virtualMachines,ukwest


In [35]:
df[['name','type','location']][df.type=='Microsoft.Network/localNetworkGateways']

,name,type,location
15,lab2-meraki-sdwan,Microsoft.Network/localNetworkGateways,westeurope


In [36]:
df[['name','type','location']][df.type=='Microsoft.Network/virtualNetworkGateways']

,name,type,location
30,lab2-vnet-gw,Microsoft.Network/virtualNetworkGateways,westeurope


In [37]:
df[['name','type','location']][df.type=='Microsoft.Network/connections']

,name,type,location
14,lab2-meraki-vpn,Microsoft.Network/connections,westeurope


In [38]:
# public IP addresses

url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/publicIPAddresses?api-version=2018-07-01'
response = requests.get(url, headers=headers)
print (response.status_code)
jsdata = response.json()
data = jsdata['value']

for item in data:
    try:
        properties = item['properties']
        print ('{:25s}{:15s}{}'.format(item['name'], item['location'], properties['ipAddress']))
    except:
        pass

200
lab2-vnet-ip             westeurope     52.178.29.168


In [39]:
# virtual networks

url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/providers/Microsoft.Network/virtualNetworks?api-version=2018-07-01'
response = requests.get(url, headers=headers)
print (response.status_code)
jsdata = response.json()
data = jsdata['value']
#print (json.dumps(data[0], indent=4))


for virtnet in data:
    properties = virtnet['properties']
    print ('virtualnet: {:20}{}'.format(virtnet['name'], properties['addressSpace']['addressPrefixes'][0]))
    subnets = properties['subnets']
    for subnet in subnets:
        print ('subnet: {:24}{}'.format(subnet['name'], subnet['properties']['addressPrefix']))
    print ()

200
virtualnet: azure-ie-vnet-01    10.1.0.0/16
subnet: azure-ie-subnet-01      10.1.0.0/24
subnet: GatewaySubnet           10.1.1.0/24

virtualnet: labsource-vnet      10.3.1.0/24
subnet: labsubnet               10.3.1.0/25
subnet: GatewaySubnet           10.3.1.128/28

virtualnet: azure-uk-vnet-04    10.8.0.0/16
subnet: azure-uk-subnet-06      10.8.0.0/24
subnet: GatewaySubnet           10.8.1.0/24

virtualnet: azure-fr-vnet-08    10.5.0.0/16
subnet: azure-fr-subnet-07      10.5.0.0/24



### Create a Resoure Group

In [40]:
import numpy as np

In [43]:
resource_group_name = 'azure-lab4-apitest-%03d' % np.random.randint(100)

url = "https://management.azure.com/subscriptions/" + azure_subscription_id + "/resourcegroups/" + resource_group_name + "?api-version=2017-05-10"
headers = { 'Authorization': 'Bearer ' + access_token, 'Content-Type': 'application/json' }
body = { 'name': resource_group_name, 'location': 'westeurope' }
response = requests.put(url, data=json.dumps(body), headers=headers)
print (response.status_code)


print (json.dumps(response.json(), indent=4))

201
{
    "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/azure-lab4-apitest-044",
    "name": "azure-lab4-apitest-044",
    "location": "westeurope",
    "properties": {
        "provisioningState": "Succeeded"
    }
}


In [44]:
# resource groups

url = "https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups?api-version=2014-04-01"
#print (url)
response = requests.get(url, headers=headers, timeout=5)
print (response.status_code)
if response.status_code!=200:
    print (response.text)
jsdata = response.json()
#print (json.dumps(jsdata['value'], indent=4))
pd.DataFrame(jsdata['value'])[['name','location']]

200


,name,location
0,azure-lab4-apitest094,westeurope
1,azure-lab4-apitest-044,westeurope
2,cloud-shell-storage-westeurope,westeurope
3,delphae-freetrail,westeurope
4,NetworkWatcherRG,westeurope


### Delete a Resoure Group

In [49]:
resource_group_name = 'azure-lab4-apitest-044' 
url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/' + resource_group_name + '?api-version=2014-04-01'
headers = { 'Authorization': 'Bearer ' + access_token, 'Content-Type': 'application/json' }
#body = { 'name': resource_group_name, 'location': 'westeurope' }
body = { 'name': resource_group_name}
response = requests.delete(url, data=json.dumps(body), headers=headers)
print (response.status_code)
print (response.text)

202



In [50]:
# ProvisioningState: Deleting

url = "https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups?api-version=2014-04-01"
#print (url)
response = requests.get(url, headers=headers, timeout=5)
print (response.status_code)
jsdata = response.json()
print (json.dumps(jsdata['value'], indent=4))
#pd.DataFrame(jsdata['value'])[['name','location']]

200
[
    {
        "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/azure-lab4-apitest-044",
        "name": "azure-lab4-apitest-044",
        "location": "westeurope",
        "properties": {
            "provisioningState": "Deleting"
        }
    },
    {
        "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/cloud-shell-storage-westeurope",
        "name": "cloud-shell-storage-westeurope",
        "location": "westeurope",
        "properties": {
            "provisioningState": "Succeeded"
        }
    },
    {
        "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail",
        "name": "delphae-freetrail",
        "location": "westeurope",
        "properties": {
            "provisioningState": "Succeeded"
        }
    },
    {
        "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/NetworkWatcherRG",
        "name": "NetworkWatcherRG",
        "location": "

### Express Route Service Providers (ERSP)

In [51]:
url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/providers/Microsoft.Network/expressRouteServiceProviders?api-version=2018-07-01'
response = requests.get(url, headers=headers)
print (response.status_code)
jsdata = response.json()
data = jsdata['value']

200


In [52]:
for ersp in data:
    if ersp['name']=='Equinix':
        print (json.dumps(ersp, indent=4))

{
    "id": "/subscriptions//resourceGroups//providers/Microsoft.Network/expressRouteServiceProviders/",
    "name": "Equinix",
    "properties": {
        "peeringLocations": [
            "Amsterdam",
            "Atlanta",
            "Chicago",
            "Dallas",
            "Dublin",
            "Hong Kong",
            "London",
            "London2",
            "Los Angeles",
            "Melbourne",
            "Miami",
            "New York",
            "Osaka",
            "Paris",
            "Sao Paulo",
            "Seattle",
            "Silicon Valley",
            "Singapore",
            "Sydney",
            "Tokyo",
            "Toronto",
            "Washington DC"
        ],
        "bandwidthsOffered": [
            {
                "offerName": "50Mbps",
                "valueInMbps": 50
            },
            {
                "offerName": "100Mbps",
                "valueInMbps": 100
            },
            {
                "offerName": "200Mbps",

In [70]:
for ersp in data:
    if 'Amsterdam' in ersp['properties']['peeringLocations']:
        #print (json.dumps(ersp, indent=4))
        values = [x['valueInMbps'] for x in ersp['properties']['bandwidthsOffered']]
        print ('{:40}{}'.format(ersp['name'], values))

Aryaka Networks                         [50, 100, 200, 500, 1000, 2000, 5000, 10000]
AT&T Netbond                            [50, 100, 200, 500, 1000, 2000, 5000, 10000]
British Telecom                         [50, 100, 200, 500, 1000, 2000, 5000, 10000]
Colt Ethernet                           [50, 100, 200, 500, 1000, 2000, 5000, 10000]
Colt IPVPN                              [50, 100, 200, 500, 1000, 2000, 5000, 10000]
Equinix                                 [50, 100, 200, 500, 1000, 2000, 5000, 10000]
euNetworks                              [50, 100, 200, 500, 1000, 2000, 5000, 10000]
GEANT                                   [50, 100, 200, 500, 1000, 2000, 5000, 10000]
InterCloud                              [50, 100, 200, 500, 1000, 2000, 5000, 10000]
InterCloud for Azure                    [50, 100, 200, 500, 1000, 2000, 5000, 10000]
Interxion                               [50, 100, 200, 500, 1000, 2000, 5000, 10000]
IX Reach                                [50, 100, 200, 500, 1000,